<a href="https://colab.research.google.com/github/utmist-numerai/William/blob/main/NN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installing requirements:

In [1]:
!pip install numerapi
!pip install halo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for halo: filename=halo-0.0.31-py3-none-any.whl size=11261 sha256=f09abe827855f4f651545866858b4d83b3aed8429a7d2cad26823c3e12ec7955
  Stored in directory: /root/.cache/pip/wheels/c0/74/6f/c63bb0075c49f05e692807fe383ddf2b17a52f567d637af096
Successfully built halo


In [3]:
import time

import keras
import numpy as np
import pandas as pd
from pathlib import Path
import json

import sklearn.metrics
import tensorflow as tf
from halo import Halo
from keras.layers import Dense, Activation, BatchNormalization
from keras import Model
from keras import Sequential
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from sklearn import svm
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split
import pickle

import numerapi
import gc
import utils_colab
from utils_colab import (
    save_model,
    load_model,
    get_biggest_change_features,
    validation_metrics,
    ERA_COL,
    DATA_TYPE_COL,
    TARGET_COL
)

# 2. Downloading and Reading data

In [4]:
start = time.time()
napi = numerapi.NumerAPI()

utils_colab.download_data()  # Training and validation
# Also get live data
current_round = napi.get_current_round(tournament=8)
napi.download_dataset("v4/live_int8.parquet", dest_path=f"./drive/MyDrive/numerai/data/live_{current_round}_int8.parquet")

print('Reading minimal training data')
# read the feature metadata and get a feature set (or all the features)
with open("./drive/MyDrive/numerai/data/features.json", "r") as f:
    feature_metadata = json.load(f)
features = feature_metadata["feature_sets"]["medium"] # get the medium feature set
# read in just those features along with era and target columns
read_columns = features + [ERA_COL, DATA_TYPE_COL, TARGET_COL]

# note: sometimes when trying to read the downloaded data you get an error about invalid magic parquet bytes...
# if so, delete the file and rerun the napi.download_dataset to fix the corrupted file
training_data = pd.read_parquet('./drive/MyDrive/numerai/data/train_int8.parquet',
                                columns=read_columns)
validation_data = pd.read_parquet('./drive/MyDrive/numerai/data/validation_int8.parquet',
                                  columns=read_columns)
live_data = pd.read_parquet(f'./drive/MyDrive/numerai/data/live_{current_round}_int8.parquet',
                                  columns=read_columns)
print('Finished reading data')

Done!


./drive/MyDrive/numerai/data/live_417_int8.parquet: 3.42MB [00:00, 5.60MB/s]                            


Reading minimal training data
Finished reading data


# 3. Parsing down training data

In [5]:
# pare down the number of eras to every 4th era
every_4th_era = training_data[ERA_COL].unique()[::4]
training_data = training_data[training_data[ERA_COL].isin(every_4th_era)]
# training_data["erano"] = training_data.era.astype(int)
X_train = training_data.filter(like='feature_', axis='columns')
Y_train = training_data[TARGET_COL]
X_val = validation_data.filter(like='feature_', axis='columns')
Y_val = validation_data[TARGET_COL]

all_feature_corrs = training_data.groupby(ERA_COL).apply(
    lambda era: era[features].corrwith(era[TARGET_COL])
)
riskiest_features = get_biggest_change_features(all_feature_corrs, 50)
gc.collect()

3439

# 4. Models

**Autoencoder for dimension reduction**

In [6]:
model_path = 'drive/MyDrive/numerai/models'

In [7]:
# Building the encoder
data_dim = len(features)
encoding_dim = 40
input_data = keras.Input(shape=(data_dim, ))
# Encoder layers
e1 = Dense(encoding_dim * 5)(input_data)
e1 = BatchNormalization()(e1)
e1 = Activation('selu')(e1)
e2 = Dense(encoding_dim)(e1)
e2 = BatchNormalization()(e2)
e2 = Activation('selu')(e2)

# bottleneck
bottleneck = Dense(encoding_dim)(e2)

# Decoder, only built for autoencoder
d1 = Dense(encoding_dim)(bottleneck)
d1 = BatchNormalization()(d1)
d1 = Activation('selu')(d1)
d2 = Dense(encoding_dim * 5)(d1)
d2 = BatchNormalization()(d2)
d2 = Activation('selu')(d2)
output = Dense(data_dim)(d2)

# Autoencoder
autoencoder = Model(input_data, output)
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.Huber())

# Separate encoder for dimension reduction later
encoder = Model(input_data, bottleneck)

es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=30,
                                      restore_best_weights=True)

X_train_tf = tf.convert_to_tensor(X_train)
X_val_tf = tf.convert_to_tensor(X_val)

autoencoder.load_weights(model_path + '/autoencoder/checkpoint')


In [ ]:
# Train the autoencoder

if model_path:
  autoencoder.load_weights(model_path + '/autoencoder/checkpoint')
  autoencoder.fit(X_train_tf, X_train_tf, epochs=30, batch_size=X_train.shape[0] // 100,
              shuffle=True, callbacks=es, validation_data=(X_val_tf, X_val_tf))
else:
  autoencoder.fit(X_train_tf, X_train_tf, epochs=30, batch_size=X_train.shape[0] // 100,
              shuffle=True, callbacks=es, validation_data=(X_val_tf, X_val_tf))


autoencoder.save_weights(model_path + '/autoencoder/checkpoint')
gc.collect()

In [8]:
X_train = encoder.predict(X_train)
X_val = encoder.predict(X_val)

75680/75680 [==============================] - 197s 3ms/step


In [9]:
print(X_train.shape)
print(X_val.shape)

(606176, 40)
(2421740, 40)


**Neural Network 1:**

In [10]:
# Removing nan rows in Y_val and their corresponding rows in X_val
# Find the indices of the NaN values in Y_validation
indices = np.isnan(Y_val)
# Remove the corresponding rows from X_validation and Y_validation
Y_val_clean = Y_val[~indices]
X_val_clean = X_val[~indices]

In [ ]:
# Training model
model_path = 'drive/MyDrive/numerai/models'

model_name = "nn_example"
NN_model = keras.models.Sequential()
input_dimension = encoding_dim

initializer = tf.keras.initializers.HeNormal()

# The Input Layer :
NN_model.add(
    keras.layers.Dense(input_dimension, kernel_initializer=initializer, input_dim=encoding_dim, activation='selu'))
NN_model.add(keras.layers.Dropout(0.2))

# The Hidden Layers :
layer_neurons = input_dimension
while layer_neurons > 4:
    if layer_neurons > 20:
        dropout = 0.2
        layer_neurons //= 2
    elif layer_neurons > 10:
        dropout = 0.1
        layer_neurons //= 3
    else:
        dropout = 0.1
        layer_neurons //= 2
    NN_model.add(keras.layers.Dense(layer_neurons, kernel_initializer=initializer,
                                    bias_regularizer=keras.regularizers.l2(1e-4)))
    NN_model.add(BatchNormalization())
    NN_model.add(Activation("selu"))
    NN_model.add(keras.layers.Dropout(dropout))

# The Output Layer :
NN_model.add(keras.layers.Dense(1, kernel_initializer='random_uniform', activation='linear'))

# Compile the network :
NN_model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])


X_train = tf.convert_to_tensor(X_train)
Y_train = tf.convert_to_tensor(Y_train)
X_val_clean = tf.convert_to_tensor(X_val_clean)
Y_val_clean = tf.convert_to_tensor(Y_val_clean)

# Early stopping callback function
esn = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1000, restore_best_weights=True)

# Train the neural network
if model_path:
    NN_model.load_weights(model_path + '/NN/checkpoint')
    NN_model.fit(X_train, Y_train, epochs=500, batch_size=X_train.shape[0] // 100, validation_data = (X_val_clean, Y_val_clean),
                 shuffle=True, callbacks=[esn])
else:
    NN_model.fit(X_train, Y_train, epochs=500, batch_size=X_train.shape[0] // 100, validation_data = (X_val_clean, Y_val_clean),
                 shuffle=True, callbacks=[esn])
    print(f'Model successfully read from {model_path}\n')


# Save weights as checkpoint
NN_model.save_weights('drive/MyDrive/numerai/models/NN/checkpoint')

gc.collect()

Epoch 1/500
101/101 [==============================] - 4s 30ms/step - loss: 0.0498 - mae: 0.1540 - mse: 0.0498 - val_loss: 0.0501 - val_mae: 0.1541 - val_mse: 0.0501
Epoch 2/500
101/101 [==============================] - 3s 30ms/step - loss: 0.0498 - mae: 0.1539 - mse: 0.0498 - val_loss: 0.0500 - val_mae: 0.1540 - val_mse: 0.0500
Epoch 3/500
101/101 [==============================] - 3s 33ms/step - loss: 0.0498 - mae: 0.1540 - mse: 0.0498 - val_loss: 0.0500 - val_mae: 0.1540 - val_mse: 0.0500
Epoch 4/500
101/101 [==============================] - 3s 27ms/step - loss: 0.0498 - mae: 0.1539 - mse: 0.0498 - val_loss: 0.0500 - val_mae: 0.1537 - val_mse: 0.0500
Epoch 5/500
101/101 [==============================] - 3s 27ms/step - loss: 0.0498 - mae: 0.1539 - mse: 0.0498 - val_loss: 0.0500 - val_mae: 0.1538 - val_mse: 0.0500
Epoch 6/500
101/101 [==============================] - 3s 27ms/step - loss: 0.0498 - mae: 0.1539 - mse: 0.0498 - val_loss: 0.0501 - val_mae: 0.1547 - val_mse: 0.0501
Epoc

2036

**Neural Network 2: to be implemented**

# 5. Predicting on validation data

In [ ]:
spinner = Halo()
spinner.start('Predicting on validation data')
# here we insert the predictions back into the validation data set, as we need to use the validation data set to
# perform feature neutralization later
X_val = tf.convert_to_tensor(X_val)
validation_data.loc[:, f"preds_{model_name}"] = NN_model.predict(X_val)
spinner.succeed()
gc.collect()

spinner.start('Neutralizing to risky features')
# neutralize our predictions to the k riskiest features in the training set
num_feature_neutralization = 200
utils_colab.neutralize_riskiest_features(training_data, validation_data, features, model_name, k=num_feature_neutralization)
spinner.succeed()
gc.collect()

print('Exporting Predictions to csv...')
validation_data["prediction"] = validation_data[f"preds_{model_name}_neutral_riskiest_{num_feature_neutralization}"] \
    .rank(pct=True)
validation_data["prediction"].to_csv(f"drive/MyDrive/numerai/predictions/NN/{model_name}.csv")
print('Done!')

print(f'Time elapsed: {(time.time() - start) / 60} mins')
gc.collect()

75680/75680 [==============================] - 179s 2ms/step
✔ Predicting on validation data
✔ Neutralizing to risky features
Exporting Predictions to csv...
Done!
Time elapsed: 180.3953409552574 mins


0